In [7]:
! pip install pymilvus llama-index python-dotenv openai transformers -q

In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

zilliz_uri = os.getenv("ZILLIZ_URI")
zilliz_token = os.getenv("ZILLIZ_TOKEN")

In [2]:
from llama_index import VectorStoreIndex, StorageContext, ServiceContext, set_global_service_context
from llama_index.callbacks import CallbackManager, TokenCountingHandler
from llama_index.vector_stores import MilvusVectorStore

In [3]:
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

# llm = Anthropic()

In [4]:
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")
# service_context = ServiceContext.from_defaults(embed_model=embed_model)

service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager, embed_model=embed_model
)

# set the global default!
# set_global_service_context(service_context)

In [5]:
vdb = MilvusVectorStore(
    uri = zilliz_uri,
    token = zilliz_token,
    collection_name = "tds_articles",
    similarity_metric = "L2",
    text_key="paragraph"
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 1063adc3165445508287c68e6a72cfe8


In [6]:
# store = StorageContext.from_defaults(vector_store=vdb)

In [7]:
vector_index = VectorStoreIndex.from_vector_store(vector_store=vdb, service_context=service_context)

In [8]:
query_engine = vector_index.as_query_engine(similarity_top_k=5)  

In [11]:
query = "What is a deep learning?"
res = query_engine.query(query)
res

Response(response='Deep learning is a type of machine learning that involves training artificial neural networks with multiple layers. It is known for its ability to automatically learn and extract complex patterns and features from large amounts of data. Deep learning has been successful in various applications such as image and speech recognition, natural language processing, and autonomous driving.', source_nodes=[NodeWithScore(node=TextNode(id_='9211801e-ee38-4728-927f-d4d1ed6ebd5d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='eb3e39b94ea1bf63bfe68391ab213af2183439e6028850bb92e00ad92c412d62', text='\u200c', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=24.830387115478516), NodeWithScore(node=TextNode(id_='6f74e688-c0c7-4998-aee7-fde0a6a9a8ac', embedding=None, metadata={}, excluded_embed_metadata_keys=[], 

In [12]:
print(token_counter.llm_token_counts[1].prompt)

system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
‌

‌‌

‍

(8)Lin, H.W., Tegmark, M. & Rolnick, D. Why Does Deep and Cheap Learning Work So Well?. J Stat Phys 168, 1223–1247 (2017). https://doi.org/10.1007/s10955-017-1836-5

InceptionNet
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What is a deep learning?
Answer: 


In [13]:
 token_counter.llm_token_counts[0].completion

'assistant: A large language model is a type of model that has the ability to perform a wide range of tasks involving language processing. These models have shown significant advancements in their capabilities, which are primarily attributed to increased computational resources and the repetition of simple operations at scale. They have evolved from being limited to narrow applications to now being able to engage in fluent, multi-turn conversations.'

In [ ]:
ids = list(res.metadata.keys())
ids

In [12]:
from pymilvus import MilvusClient

client = MilvusClient(
uri=zilliz_uri,
token=zilliz_token,
)


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: cf0dddc733e940ff979bd6fd499ed475


In [38]:
client.get(
    collection_name="tds_articles",
    ids='https://towardsdatascience.com/how-to-one-way-anova-by-hand-4c19e2a61a8c+6',
    output_fields=["paragraph"]
)


[{'paragraph': 'Note that the ANOVA requires some assumptions (i.e., independence, equality of variances and normality). The aim of this post is to illustrate how to do an ANOVA by hand and not how to verify these assumptions, so we suppose they are met without any verification. See how to test these assumptions in R if you are interested.',
  'id': 'https://towardsdatascience.com/how-to-one-way-anova-by-hand-4c19e2a61a8c+6'}]

In [16]:
!python -V

Python 3.10.12


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
